In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import os

os.chdir("/content/drive/My Drive/Colab_Notebooks/ANNClassification")

In [3]:
!ls

app.py			  Model.h5	       requirements.txt  untitled_project
Churn_Modelling.csv	  one_hot_encoder.pkl  scaler.pkl
label_encoder_gender.pkl  prediction.ipynb     training.ipynb


In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [5]:
## load trained model, scaler.pkl, encoder.pkl

model = load_model('Model.h5')

## load encoder and scaler

with open('label_encoder_gender.pkl', 'rb') as file:
  label_encoder_gender = pickle.load(file)

with open('one_hot_encoder.pkl', 'rb') as f:
  one_hot_encoder_geo = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
  scaler = pickle.load(f)

In [6]:
# example input data

input_data = {
    'CreditScore': 653,
    'Geography': 'Germany',
    'Gender': 'Male',
    'Age': 58,
    'Tenure': 1,
    'Balance': 132602.88,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 0,
    'EstimatedSalary': 5097.67

}

In [7]:
geo_encoded = one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [8]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,653,Germany,Male,58,1,132602.88,1,1,0,5097.67


In [9]:
## encode categorical var

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,653,Germany,1,58,1,132602.88,1,1,0,5097.67


In [10]:
## conbine one hot encoded col with input df

input_df = pd.concat([input_df.drop(['Geography'], axis=1),geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,653,1,58,1,132602.88,1,1,0,5097.67,0.0,1.0,0.0


In [11]:
## scaling data

input_scaled = scaler.transform(input_df)
input_scaled

array([[ 0.01403458,  0.91324755,  1.81609745, -1.38944225,  0.90463276,
        -0.91668767,  0.64920267, -1.02583358, -1.65753422, -0.99850112,
         1.72572313, -0.57638802]])

In [12]:
## prediction

prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step


array([[0.9999974]], dtype=float32)

In [13]:
prediction_prob = prediction[0][0]
prediction_prob

0.9999974

In [14]:
if prediction_prob > 0.5:
  print('Customer will leave the bank')
else:
  print('Customer will not leave the bank')

Customer will leave the bank


In [15]:
!pip install gradio

In [17]:
!python app.py


2025-02-27 08:13:10.019937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740643990.077603   14531 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740643990.094994   14531 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 08:13:16.217674: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://019cc8b206805ec98d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working director